1.如何实现可迭代对象和迭代器对象？
数据抓取较大时，数据应采用，用时访问？，抓取一条存储显示一条。

可迭代对象和迭代器对象转化iter(list) 
可迭代对象内部调用了list.__iter__()方法

collections下已经封装了Iterable，Itrator
可以使用.__abstractmethods__ 查看其使用接口是什么

In [19]:
import requests

def getWeather(city):
    url = 'http://wthrcdn.etouch.cn/weather_mini?city=' + city
    r = requests.get(url)
    data = r.json()['data']['forecast'][0]
    return '%s:%s,%s'%(city,data['low'],data['high'])
getWeather('深圳')

'深圳:低温 22℃,高温 27℃'

In [2]:
from collections import Iterable,Iterator
import requests

class WeatherIterator(Iterator):
    def __init__(self, cities):
        self.cities = cities
        self.index = 0
    
    def getWeather(self, city):
        url = 'http://wthrcdn.etouch.cn/weather_mini?city=' + city
        r = requests.get(url)
        data = r.json()['data']['forecast'][0]
        return '%s:%s,%s'%(city,data['low'],data['high'])

    def __next__(self):
        if self.index == len(self.cities):
            raise StopIteration
        city = self.cities[self.index]
        self.index += 1
        return self.getWeather(city)


class WeatherIterable(Iterable):
    def __init__(self,cities):
        self.cities = cities
        
    def __iter__(self):
        return WeatherIterator(self.cities)
 
city_list =['深圳','广州','珠海']
for x in WeatherIterable(city_list):
    print(x)  

深圳:低温 22℃,高温 27℃
广州:低温 19℃,高温 26℃
珠海:低温 21℃,高温 28℃


2.如何使用生成器函数实现可迭代对对象？实现一个可迭代对象的类，它能迭代出给定范围的所有素数

包含yield语句的函数为生成器函数。
g = f()  g生成器对象本身是可迭代对象,g.__iter__() 返回自身即生成器对象

g.__iter__() is g  #True

In [3]:
def f():
    i = 2
    yield i
g = f()

In [15]:
class PrimeNumbers(object):
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
    def isPrimeNum(self, k):
        if k < 2:
            return False
        for i in range(2,k):
            if k % i == 0:
                return False
            return True
    
    def __iter__(self):
        for k in range(self.start, self.end+1):
            if self.isPrimeNum(k):
                yield k
for x in PrimeNumbers(1,100):
    print(x)

3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
91
93
95
97
99


3.如何进行反向迭代以及如何实现反向迭代？如
常规方法：对一个列表l=[1,2,3,4,5]反向迭代
1. l.reverse()先反向再迭代，修改了原列表
2. l[::-1] 切片，但生成了新列表，浪费空间

最佳：
用reversed()方法,得到列表反向迭代器，和iter(l)相反，内部调用l.__reversed__()方法

In [23]:
class FloatRange(object):
    def __init__(self, start, end, step=0.1):
        self.start = start
        self.end = end
        self.step = step
    def __iter__(self):
        t = self.start
        while t <= self.end:
            yield t
            t += self.step
            
    def __reversed__(self):
        t = self.end
        while t >= self.start:
            yield t
            t -=self.step
for x in FloatRange(1.0,4.0,0.5):
    print(x)
    
for x in reversed(FloatRange(1.0,4.0,0.5)):
    print(x)

1.0
1.5
2.0
2.5
3.0
3.5
4.0


TypeError: argument to reversed() must be a sequence

5.如何对迭代器做切片操作？
读取某文本文件100～300行之间的内容，python文本为可迭代对象，可否使用类似的列表切片方式得到指定内容的生成器？
f = open('a.txt','r')
f[100:300]??
f.readlines() #一次把文件全都读取

f.seek(0) # 返回文件头部
for line in f：
    print(line)
   
使用itertools下的islice，但本身会改变文件的读取指针，会消耗原来的迭代对象，每次使用时重新复制文件

In [24]:
from itertools import islice
islice(f,100,300) # 100到300行
islice(f,100,None) # 100到最后
for line in islice(f, 100, 300):
    print(line)

NameError: name 'islice' is not defined

如何在一个for语句中迭代多个可迭代对象？
并行 同时合并可迭代列表
串行 先迭代一个，再迭代一个，itertools.chain

In [41]:
from random import randint
math = [randint(60,100) for _ in range(10)]
english = [randint(60,100) for _ in range(10)]

for i in range(len(math)-1):
    print(math[i] + english[i]) # 使用引索


168
157
178
158
143
182
181
161
153


In [43]:
# zip, 如果数量不等，取最短的输出
math = [randint(60,100) for _ in range(10)]
english = [randint(60,100) for _ in range(10)]
for x,y in zip(math,english):
    print(x,y)
zip(math,english)



100 80
93 77
99 71
85 78
62 80
71 75
88 70
86 73
88 99
79 79


In [48]:
#itertools
from itertools import chain
math = [randint(60,100) for _ in range(10)]
english = [randint(60,100) for _ in range(10)]
count = 0
for x in chain(math,english):
    # 选出单科90的人数
    if x > 90:
        count += 1
print(count)

7
